In [29]:
import numpy as np
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.mappers import JordanWignerMapper

In [24]:
n = 10
N = 2*n
t=1
mu=1
U=1

In [25]:
op = FermionicOp(
    {
        "+_0 -_0": 1.0,
        "+_1 -_1": -1.0,
    },
    num_spin_orbitals=2,
)

In [26]:
print(op)

Fermionic Operator
number spin orbitals=2, number terms=2
  1.0 * ( +_0 -_0 )
+ -1.0 * ( +_1 -_1 )


In [27]:
def qOp(i,j,N=20):
    return FermionicOp(
    {
        "+_{i} -_{j}".format(i=i%N,j=j%N): 1.0,
    },
    num_spin_orbitals=N,
)

In [28]:
t_term = 0
mu_term =0
U_term =0
for i in range(n):
    t_term+= qOp(2*i,2*i+2) + qOp(2*i+2,2*i) + qOp(2*i+1,2*i+3) + qOp(2*i+3,2*i+1) 
    mu_term += qOp(2*i,2*i) + qOp(2*i+1,2*i+1)
    U_term += (qOp(2*i,2*i) @ qOp(2*i+1, 2*i+i))
H = U * U_term - mu * mu_term - t * t_term

print(H)

Fermionic Operator
number spin orbitals=20, number terms=70
  1.0 * ( +_0 -_0 +_1 -_0 )
+ 1.0 * ( +_2 -_2 +_3 -_3 )
+ 1.0 * ( +_4 -_4 +_5 -_6 )
+ 1.0 * ( +_6 -_6 +_7 -_9 )
+ 1.0 * ( +_8 -_8 +_9 -_12 )
+ 1.0 * ( +_10 -_10 +_11 -_15 )
+ 1.0 * ( +_12 -_12 +_13 -_18 )
+ 1.0 * ( +_14 -_14 +_15 -_1 )
+ 1.0 * ( +_16 -_16 +_17 -_4 )
+ 1.0 * ( +_18 -_18 +_19 -_7 )
+ -1.0 * ( +_12 -_12 )
+ -1.0 * ( +_9 -_9 )
+ -1.0 * ( +_5 -_5 )
+ -1.0 * ( +_10 -_10 )
+ -1.0 * ( +_15 -_15 )
+ -1.0 * ( +_4 -_4 )
+ -1.0 * ( +_0 -_0 )
+ -1.0 * ( +_7 -_7 )
+ -1.0 * ( +_3 -_3 )
+ -1.0 * ( +_18 -_18 )
+ -1.0 * ( +_8 -_8 )
+ -1.0 * ( +_13 -_13 )
+ -1.0 * ( +_17 -_17 )
+ -1.0 * ( +_14 -_14 )
+ -1.0 * ( +_2 -_2 )
+ -1.0 * ( +_1 -_1 )
+ -1.0 * ( +_6 -_6 )
+ -1.0 * ( +_16 -_16 )
+ -1.0 * ( +_11 -_11 )
+ -1.0 * ( +_19 -_19 )
+ -1.0 * ( +_19 -_1 )
+ -1.0 * ( +_13 -_11 )
+ -1.0 * ( +_12 -_14 )
+ -1.0 * ( +_18 -_0 )
+ -1.0 * ( +_11 -_13 )
+ -1.0 * ( +_7 -_9 )
+ -1.0 * ( +_17 -_15 )
+ -1.0 * ( +_10 -_12 )
+ -1.0 * ( +_2 -_4 )
+

In [30]:
mapper = JordanWignerMapper()
qubit_jw_op = mapper.map(H)
print(qubit_jw_op)

SparsePauliOp(['IIIIIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIZIII', 'IIIIIIIIIIIIIIIIIZII', 'IIIIIIIIIIIIIIIIZZII', 'IIIIIIIIIIIIIXYIIIII', 'IIIIIIIIIIIIIYYIIIII', 'IIIIIIIIIIIIIXXIIIII', 'IIIIIIIIIIIIIYXIIIII', 'IIIIIIIIIIIIIXYZIIII', 'IIIIIIIIIIIIIYYZIIII', 'IIIIIIIIIIIIIXXZIIII', 'IIIIIIIIIIIIIYXZIIII', 'IIIIIIIIIIXZYIIIIIII', 'IIIIIIIIIIYZYIIIIIII', 'IIIIIIIIIIXZXIIIIIII', 'IIIIIIIIIIYZXIIIIIII', 'IIIIIIIIIIXZYZIIIIII', 'IIIIIIIIIIYZYZIIIIII', 'IIIIIIIIIIXZXZIIIIII', 'IIIIIIIIIIYZXZIIIIII', 'IIIIIIIXZZYIIIIIIIII', 'IIIIIIIYZZYIIIIIIIII', 'IIIIIIIXZZXIIIIIIIII', 'IIIIIIIYZZXIIIIIIIII', 'IIIIIIIXZZYZIIIIIIII', 'IIIIIIIYZZYZIIIIIIII', 'IIIIIIIXZZXZIIIIIIII', 'IIIIIIIYZZXZIIIIIIII', 'IIIIXZZZYIIIIIIIIIII', 'IIIIYZZZYIIIIIIIIIII', 'IIIIXZZZXIIIIIIIIIII', 'IIIIYZZZXIIIIIIIIIII', 'IIIIXZZZYZIIIIIIIIII', 'IIIIYZZZYZIIIIIIIIII', 'IIIIXZZZXZIIIIIIIIII', 'IIIIYZZZXZIIIIIIIIII', 'IXZZZZYIIIIIIIIIIIII', 'IYZZZZYIIIIIIIIIIIII', 'IXZZZZXIIIIIIIIIIIII', 'IYZZZZXIIIIIIIIIIIII', 'IXZZZZYZIIIIIIIIIIII', '

In [ ]:
:)